In [ ]:
# Install these libraries
!pip install selenium
!pip install pywhatkit
!pip install pyautogui
!pip install keyboard

In [ ]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests

import pywhatkit as kit
import datetime

import time 
import pyautogui 
import keyboard as k 

In [ ]:
# Replace all the following with your respective details

# Replace with the course of your choice in capital letters - string format.
#Eg: course="CSE" or course="AMS", or others, etc.
course="CSE"

# Replace with the course subject id's you want to track - list of string format. 
#Eg interested_courses=['508','564','551', '527']
interested_courses=['508','564','551']

# Solar id (Replace with your id) - number format
sbu_net_id=123456789

# Solar password (Replace with your password) - string format
sbu_password="YourPassword"

# Replace with the whatsApp group ending code for a whatsapp group shared link, to which you want to send the updates.
# Eg: if whatsapp gorup link is https://chat.whatsapp.com/abcxxxxxxxxxxxxxxxxyz
# Then use whatsapp_group_id="abcxxxxxxxxxxxxxxxxyz" - string format
whatsapp_group_id="xxxxxxxxxxxxxxxxxxxxx"

In [ ]:
# get authentication cookies from solar login
def get_cookies_func(sbu_net_id,sbu_password):
    #start selenium driver
    driver=webdriver.Chrome()

    #launch main URL
    driver.get("https://prod.ps.stonybrook.edu/psp/csprods/EMPLOYEE/CAMP/h/?tab=DEFAULT&cmd=login&errorCode=105&languageCd=ENG")

    #press username and give input
    username = driver.find_element(By.ID, "userid")
    username.send_keys(sbu_net_id)

    #press password and give input
    password = driver.find_element(By.ID, "pwd")
    password.send_keys(sbu_password)

    #press login button
    loginbutton = driver.find_element(By.CLASS_NAME, "psloginbutton")
    loginbutton.click()

    sleep(1)

    #press enrollment button
    enrollment = driver.find_element(By.LINK_TEXT, "Enrollment")
    enrollment.click()

    #launch enrollment page
    driver.get("https://prod.ps.stonybrook.edu/psp/csprods/EMPLOYEE/CAMP/c/PRJCS_MENU.PRJCS_SCHD_STRT.GBL?PORTALPARAM_PTCNAV=PRJCS_SCHD_STRT&EOPP.SCNode=CAMP&EOPP.SCPortal=EMPLOYEE&EOPP.SCName=ADMN_SOLAR_SYSTEM&EOPP.SCLabel=Enrollment&EOPP.SCFName=HCCC_ENROLLMENT&EOPP.SCSecondary=true&EOPP.SCPTcname=PT_PTPP_SCFNAV_BASEPAGE_SCR&FolderPath=PORTAL_ROOT_OBJECT.CO_EMPLOYEE_SELF_SERVICE.SU_STUDENT_FOLDER.HCCC_ENROLLMENT.PRJCS_SCHD_STRT&IsFolder=false")

    #launch enrollment page with actual portal url
    driver.get("https://prod.ps.stonybrook.edu/psc/csprods/EMPLOYEE/CAMP/c/PRJCS_MENU.PRJCS_SCHD_STRT.GBL?PORTALPARAM_PTCNAV=PRJCS_SCHD_STRT&EOPP.SCNode=CAMP&EOPP.SCPortal=EMPLOYEE&EOPP.SCName=ADMN_SOLAR_SYSTEM&EOPP.SCLabel=Enrollment&EOPP.SCFName=HCCC_ENROLLMENT&EOPP.SCSecondary=true&EOPP.SCPTcname=PT_PTPP_SCFNAV_BASEPAGE_SCR&FolderPath=PORTAL_ROOT_OBJECT.CO_EMPLOYEE_SELF_SERVICE.SU_STUDENT_FOLDER.HCCC_ENROLLMENT.PRJCS_SCHD_STRT&IsFolder=false&PortalActualURL=https%3a%2f%2fprod.ps.stonybrook.edu%2fpsc%2fcsprods%2fEMPLOYEE%2fCAMP%2fc%2fPRJCS_MENU.PRJCS_SCHD_STRT.GBL&PortalContentURL=https%3a%2f%2fprod.ps.stonybrook.edu%2fpsc%2fcsprods%2fEMPLOYEE%2fCAMP%2fc%2fPRJCS_MENU.PRJCS_SCHD_STRT.GBL&PortalContentProvider=CAMP&PortalCRefLabel=Schedule%20Builder&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fprod.ps.stonybrook.edu%2fpsp%2fcsprods%2f&PortalURI=https%3a%2f%2fprod.ps.stonybrook.edu%2fpsc%2fcsprods%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes")

    #press schedule builder
    schedule_builder = driver.find_element(By.ID, "win0divPRJCS_DERIVED_PRJCS_LAUNCH_CS")
    schedule_builder.click()

    #get login cookies for stonybrook.collegescheduler.com domain
    driver.get("https://stonybrook.collegescheduler.com/entry")
    cookies_list=driver.get_cookies()

    #stop selenium driver
    driver.quit()
    
    # return list of cookies
    return cookies_list

In [ ]:
cookies_list=[]

#infinite while loop
while 1:
    if not cookies_list:
        cookies_list=get_cookies_func(sbu_net_id,sbu_password)

    cookie_value=""

    for cookie_name in cookies_list:
        if cookie_name['name']=='.AspNet.Cookies':
            cookie_value=cookie_name['value']

    cookies=".AspNet.Cookies=" + cookie_value
    
    headers={
        'cookie': cookies,
        'content-type': 'text/plain;charset=UTF-8',
        'connection':'keep-alive',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    print("Got new cookies")
    
    #get new cookies after every 15 times, as cookies expire and need to be replaced by new authentication cookies
    i=0
    while i<15:
        r = requests.get('https://stonybrook.collegescheduler.com/api/terms/', headers=headers)

        term=r.json()[-1].get('id')

        req = requests.get('https://stonybrook.collegescheduler.com/api/terms/'+term+'/Subjects/'+course+'/Courses/', headers=headers)

        open_courses_res=""

        for r in req.json():
            course_number=r['number']   

            if course_number in interested_courses:
                final_req = requests.get('https://stonybrook.collegescheduler.com/api/terms/'+term+'/Subjects/'+course+'/Courses/'+course_number+'/regblocks', headers=headers)
                
                # Check for open seats in interested_courses
                if final_req.json()['sections'] and final_req.json()['sections'][0]['openSeats']>0:
                    open_seats=final_req.json()['sections'][0]['openSeats']
                    MessageString=str(open_seats)+" seats open in:"+r['displayTitle']
                    print("Loop count "+str(i)+" , "+MessageString)
                    open_courses_res+=MessageString+"                 "
        
        # Send notification in whatsapp group
        if len(open_courses_res)>0:
            kit.sendwhatmsg_to_group_instantly(whatsapp_group_id, open_courses_res)
            
            # Change X,Y values in pyautogui.click(1475, 645), if the message is automatically typed in whatsaspp group chat, but the text is not getting submitted.
            # Find the approx coordinates of submit button using mouse coordinate extension - https://chrome.google.com/webstore/detail/mouse-coordinates/mfohnjojhopfcahiddmeljeholnciakl
            pyautogui.click(1475, 645) 
            time.sleep(2) 
            k.press_and_release('enter')

            k.press_and_release('ctrl+w')
        else:
            print("Loop count "+str(i)+" , "+"No seats in your selected courses list")

        
        i+=1
        sleep(45)
        
    cookies_list=[]